# Imports

In [1]:
import os
import supervisely_lib as sly

# Initialize API access with your credentials

In [2]:
address = os.environ['SERVER_ADDRESS']
token = os.environ['API_TOKEN']

print("Server address: ", address)
print("Your API token: ", token)

# Initialize the API access object.
api = sly.Api(address, token)

Server address:  http://192.168.1.69:5555
Your API token:  HfQ2owV8QjwojwnTiaPzIyEZtncIBjISnQqgBzKmDTjTL6WmV80kbd9J5DHu8PnCPVBqWBUXcOQlqjUBiCrQuUBxh562iaqAzqa4z80lJYjvxTFky5RbHDXregjOf2y8


# Script parameters

In [3]:
team_name = "max"
workspace_name = "First Workspace"
project_name = "tutorial_project"  # will be clonned from Explore page

# Verify parameters

In [4]:
team = api.team.get_info_by_name(team_name)
if team is None:
    raise RuntimeError("Team {!r} not found".format(team_name))

workspace = api.workspace.get_info_by_name(team.id, workspace_name)
if workspace is None:
    raise RuntimeError("Workspace {!r} not found".format(workspace_name))

print("Team: id={}, name={}".format(team.id, team.name))
print("Workspace: id={}, name={}".format(workspace.id, workspace.name))

Team: id=4, name=max
Workspace: id=7, name=First Workspace


# Clone demo project from Explore page

In [5]:
project = api.project.get_info_by_name(workspace.id, project_name)
if project is None:
    print("Project {!r} not found, will be clonned from explore".format(project_name))
else:
    print('Project with the same name {!r} already exists, '
          'demo project will be clonned from Explore page and saved to with the new name'.format(project_name))
    project_name = api.project.get_free_name(workspace.id, project_name)
task_id = api.project.clone_from_explore('Supervisely/Demo/tutorial_project', workspace.id, project_name)
api.task.wait(task_id, api.task.Status.FINISHED)
project = api.project.get_info_by_name(workspace.id, project_name)
print("Project (id={}, name={}) has been sucessfully cloned from explore".format(project.id, project.name))

Project 'tutorial_project' not found, will be clonned from explore
Project (id=510, name=tutorial_project) has been sucessfully cloned from explore


# Helper functions

In [6]:
def print_compact_project_structure(project):
    datasets = api.dataset.get_list(project.id)
    print("\nProject {!r} contains {} datasets:".format(project.name, len(datasets)))
    for dataset in datasets:
        print("Id: {:<5} Name: {:<25s} images count: {:<5}".format(dataset.id, dataset.name, dataset.images_count))

# Use cases: copy/move/delete

## Print original project structure

In [7]:
print_compact_project_structure(project)


Project 'tutorial_project' contains 2 datasets:
Id: 1581  Name: dataset_01                images count: 3    
Id: 1582  Name: dataset_02                images count: 2    


## Copy dataset

In [8]:
dataset = api.dataset.get_info_by_name(project.id, 'dataset_01')
copied_dataset = api.dataset.copy(project.id, dataset.id, 'dataset_01_copied', change_name_if_conflict=True, with_annotations=True)
print_compact_project_structure(project)


Project 'tutorial_project' contains 3 datasets:
Id: 1581  Name: dataset_01                images count: 3    
Id: 1582  Name: dataset_02                images count: 2    
Id: 1583  Name: dataset_01_copied         images count: 3    


## Copy images from one dataset to another

In [9]:
src_dataset = api.dataset.get_info_by_name(project.id, 'dataset_01')
dst_dataset = api.dataset.get_info_by_name(project.id, 'dataset_02')

src_images = api.image.get_list(src_dataset.id)
src_image_ids = [image.id for image in src_images]
new_images = api.image.copy_batch(dst_dataset.id, src_image_ids, change_name_if_conflict=True, with_annotations=True)

print_compact_project_structure(project)


Project 'tutorial_project' contains 3 datasets:
Id: 1581  Name: dataset_01                images count: 3    
Id: 1582  Name: dataset_02                images count: 5    
Id: 1583  Name: dataset_01_copied         images count: 3    


## Move some images to new dataset

In [10]:
src_dataset = api.dataset.get_info_by_name(project.id, 'dataset_02')
dst_dataset = api.dataset.create(project.id, 'moved_images')

all_dataset_images = api.image.get_list(src_dataset.id)
image_ids_to_move = [image.id for image in all_dataset_images][:3]

new_images = api.image.move_batch(dst_dataset.id, image_ids_to_move, change_name_if_conflict=True, with_annotations=False)

print_compact_project_structure(project)


Project 'tutorial_project' contains 4 datasets:
Id: 1581  Name: dataset_01                images count: 3    
Id: 1582  Name: dataset_02                images count: 2    
Id: 1583  Name: dataset_01_copied         images count: 3    
Id: 1584  Name: moved_images              images count: 3    


## Delete images from dataset

In [11]:
src_dataset = api.dataset.get_info_by_name(project.id, 'moved_images')
all_dataset_images = api.image.get_list(src_dataset.id)

image_ids_to_remove = [image.id for image in all_dataset_images][:2]
api.image.remove_batch(image_ids_to_remove)

print_compact_project_structure(project)


Project 'tutorial_project' contains 4 datasets:
Id: 1581  Name: dataset_01                images count: 3    
Id: 1582  Name: dataset_02                images count: 2    
Id: 1583  Name: dataset_01_copied         images count: 3    
Id: 1584  Name: moved_images              images count: 1    


## Remove datasets from project

In [12]:
dataset1 = api.dataset.get_info_by_name(project.id, 'moved_images')
dataset2 = api.dataset.get_info_by_name(project.id, 'dataset_01_copied')

ids_to_remove = [dataset1.id, dataset2.id]
api.dataset.remove_batch(ids_to_remove)

print_compact_project_structure(project)


Project 'tutorial_project' contains 2 datasets:
Id: 1581  Name: dataset_01                images count: 3    
Id: 1582  Name: dataset_02                images count: 2    


## Remove entire demo project

In [13]:
api.project.remove(project.id)
project = api.project.get_info_by_name(workspace.id, project_name)
if project is None:
    print('Project {!r} has been successfully removed'.format(project_name))

Project 'tutorial_project' has been successfully removed
